In [1]:
import os
import csv
import numpy as np
import math
import heapq
import pandas as pd
#directory = "../data/101/"

In [2]:
def readSmartphone():
    path = directory+'phone/'+str(currentid)+"_"+str(currentblock)+"_calibration_1234.csv"
    smartphonelog = []
    # see if the file exist
    if os.path.isfile(path):
        f = open(path);
        reader = csv.reader(f)
        flag = 0
        for line in reader:
            flag = flag + 1
            if flag > 1:
                smartphonelog.append(line)
        f.close()
    return smartphonelog

In [3]:
def readKeyevent():
    path = directory+"key_event_"+str(currentid)+"_"+str(currentblock)+".csv"
    smartphonelog = []
    # see if the file exist
    if os.path.isfile(path):
        f = open(path);
        reader = csv.reader(f)
        flag = 0
        for line in reader:
            flag = flag + 1
            if flag > 1:
                smartphonelog.append(line)
        f.close()
    return smartphonelog

In [4]:
def addtrialtime(smartphonelog):
    starttime = int(float(smartphonelog[0][0]))
    #print(starttime)
    for line in smartphonelog:
        line.append(int(float(line[0])) - starttime)
    return smartphonelog

In [5]:
#readSmartphone()

In [6]:
#addtrialtime(readSmartphone())

In [7]:
def readOptiTrack():
    path = directory+'optitrack/'+str(currentid)+"_"+str(currentblock)+"_optitrack.csv"
    optitracklog = []
    # see if the file exist
    if os.path.isfile(path):
        f = open(path);
        reader = csv.reader(f)
        flag = 0
        for line in reader:
            flag = flag + 1
            flagdataexist = 1
            for item in line:
                if item=='':
                    flagdataexist = 0
            if flag > 7 and flagdataexist==1:
                #在readOptiTrack的fps=240的情况下，这样的处理使fps下降到60
                optitracklog.append(line)
        f.close()
    if len(optitracklog[0])==17:
        #识别使用手指的数量
        hand = 1
    elif len(optitracklog[0])==20:
        hand = 2
    else:
        print('Strange marker number: '+str((len(optitracklog)-2)/3))
    return [hand,optitracklog]

In [8]:
def readEnterlog():
    path = directory+'phone/'+str(currentid)+"_"+str(currentblock)+"_calibration_enter.csv"
    enterlog = []
    # see if the file exist
    if os.path.isfile(path):
        f = open(path);
        reader = csv.reader(f)
        flag = 0
        for line in reader:
            flag = flag + 1
            if flag > 1:
                enterlog.append(line)
        f.close()
    return enterlog
    

In [9]:
def generateAllPoints(optitracklog, hand):
    
    point1 = [[],[],[]]
    point2 = [[],[],[]]
    point3 = [[],[],[]]
    point4 = [[],[],[]]
    point5 = [[],[],[]]
    point6 = [[],[],[]]
    for line in optitracklog:
        #写入x值
        point1[0].append(float(line[2]))
        point2[0].append(float(line[5]))
        point3[0].append(float(line[8]))
        point4[0].append(float(line[11]))
        point5[0].append(float(line[14]))

        #写入y值
        point1[1].append(float(line[3]))
        point2[1].append(float(line[6]))
        point3[1].append(float(line[9]))
        point4[1].append(float(line[12]))
        point5[1].append(float(line[15]))

        #写入z值
        point1[2].append(float(line[4]))
        point2[2].append(float(line[7]))
        point3[2].append(float(line[10]))
        point4[2].append(float(line[13]))
        point5[2].append(float(line[16]))
        
        if hand==2:
            #如果两只手则向point6添加x,y,z坐标信息
            point6[0].append(float(line[17]))
            point6[1].append(float(line[18]))
            point6[2].append(float(line[19]))

    if hand==1:
        allpoints = [point1, point2, point3, point4, point5]
    elif hand==2:
        allpoints = [point1, point2, point3, point4, point5, point6]
    return allpoints

def getIndex(allpoints, hand):
    #确定手指数据的index
    z1 = allpoints[0][2]
    z2 = allpoints[1][2]
    z3 = allpoints[2][2]
    z4 = allpoints[3][2]
    z5 = allpoints[4][2]
    if hand==2:
        z6 = allpoints[5][2]
    #利用所有marker在z轴（指向墙方向）上的平均值，找出4个marker（定位marker的z坐标总是大于手指的z坐标）
    if hand==1:
        zmean = [np.mean(z1),np.mean(z2),np.mean(z3),np.mean(z4),np.mean(z5)]
        fingerIndex = list(map(zmean.index, heapq.nsmallest(1, zmean)))[0]+1
        print('FingerIndex = '+str(fingerIndex))
    elif hand==2:
        zmean = [np.mean(z1),np.mean(z2),np.mean(z3),np.mean(z4),np.mean(z5),np.mean(z6)]
        finger1Index = list(map(zmean.index, heapq.nsmallest(2, zmean)))[0]+1
        finger2Index = list(map(zmean.index, heapq.nsmallest(2, zmean)))[1]+1
        print('Finger1Index = '+str(finger1Index))
        print('Finger2Index = '+str(finger2Index))

    #确定4个marker的位置
    middlepointx = 0
    middlepointy = 0
    middlepointz = 0
    if hand==1:
        markerIndex = [1,2,3,4,5]
        for i in markerIndex:
            if i==fingerIndex:
                continue
            else:
                middlepointx = middlepointx + allpoints[i-1][0][0]
                middlepointy = middlepointy + allpoints[i-1][1][0]
                middlepointz = middlepointz + allpoints[i-1][2][0]
    elif hand==2:
        markerIndex = [1,2,3,4,5,6]
        for i in markerIndex:
            if i==finger1Index or i==finger2Index:
                continue
            else:
                middlepointx = middlepointx + allpoints[i-1][0][0]
                middlepointy = middlepointy + allpoints[i-1][1][0]
                middlepointz = middlepointz + allpoints[i-1][2][0]
    
    #4个marker构成矩形的中点坐标 = 4个marker坐标之和/4
    middlepointx = middlepointx/4
    middlepointy = middlepointy/4
    middlepointz = middlepointz/4
    #确定marker位置与其index之间的关系
    #marker1,2,3,4的位置分别为左上，右上，左下，右下
    #optiTrack的坐标方向为：xoz在桌面上，x向左（实验者左手方向）为正，z向前（墙面方向）为正，y向上（桌面上方）为正
    if hand==1:
        for i in markerIndex:
            if i==fingerIndex:
                continue
            else:
                if allpoints[i-1][0][0]<middlepointx and allpoints[i-1][2][0]>middlepointz:
                    marker2index = i
                elif allpoints[i-1][0][0]>middlepointx and allpoints[i-1][2][0]>middlepointz:
                    marker1index = i
                elif allpoints[i-1][0][0]<middlepointx and allpoints[i-1][2][0]<middlepointz:
                    marker4index = i
                elif allpoints[i-1][0][0]>middlepointx and allpoints[i-1][2][0]<middlepointz:
                    marker3index = i
        indexlist = [fingerIndex,marker1index,marker2index,marker3index,marker4index]
    elif hand==2:
        for i in markerIndex:
            if i==finger1Index or i==finger2Index:
                continue
            else:
                if allpoints[i-1][0][0]<middlepointx and allpoints[i-1][2][0]>middlepointz:
                    marker2index = i
                elif allpoints[i-1][0][0]>middlepointx and allpoints[i-1][2][0]>middlepointz:
                    marker1index = i
                elif allpoints[i-1][0][0]<middlepointx and allpoints[i-1][2][0]<middlepointz:
                    marker4index = i
                elif allpoints[i-1][0][0]>middlepointx and allpoints[i-1][2][0]<middlepointz:
                    marker3index = i
        indexlist = [finger1Index,marker1index,marker2index,marker3index,marker4index,finger2Index]
    
    #计算OptiTrack，smartphone和 mm的换算
    # mmPerOpt =1018.4287099903007
    # pixelPerOpt = 23278.370514064016
    
    mmPerOpt = 63/abs(allpoints[marker1index-1][0][0]-allpoints[marker2index-1][0][0])
    pixelPerOpt = 1440/abs(allpoints[marker1index-1][0][0]-allpoints[marker2index-1][0][0])
    print('每OptiTrack单位对应mm数为'+str(mmPerOpt))
    print('每OptiTrack单位对应手机pixel数为'+str(pixelPerOpt))
    
    return indexlist


In [10]:
def showAllpoints(allpoints,hand):
    #三维可视化所有marker的坐标（不同颜色区分）
    import numpy as np
    import matplotlib.pyplot as plt
    from mpl_toolkits.mplot3d import Axes3D

    #创建一个三维的绘图工程
    ax = plt.subplot(111, projection='3d')

    #5个marker以不同的颜色画出
    ax.scatter(allpoints[0][0],allpoints[0][1],allpoints[0][2], c='y')
    ax.scatter(allpoints[1][0],allpoints[1][1],allpoints[1][2], c='r')
    ax.scatter(allpoints[2][0],allpoints[2][1],allpoints[2][2], c='g')
    ax.scatter(allpoints[3][0],allpoints[3][1],allpoints[3][2], c='b')
    ax.scatter(allpoints[4][0],allpoints[4][1],allpoints[4][2], c='c')
    if hand ==2:
        ax.scatter(allpoints[5][0],allpoints[5][1],allpoints[5][2], c='#FE2EF7')
    
    #ax.scatter(middlepointx,middlepointy,middlepointz, c='#FE2EF7')

    ax.set_zlabel('Z')
    ax.set_ylabel('Y')
    ax.set_xlabel('X')
    plt.show()

In [11]:
def optTomm(optlength):
    mmlength = optlength * 1018.4
    return mmlength
    
def optToPixel(optlength):
    pixellength = optlength * 23278.4
    return pixellength

def pixelTomm(pixellength):
    mmlength = (1018.4*pixellength)/23278.4
    return mmlength

def mmTopixel(mmlength):
    pixellength = (23278.4*mmlength)/1018.4
    return pixellength

In [12]:
def transferOptiToPixel(allpoints):
    #把当前的optiTrack坐标单位转换成以pixel为单位的坐标
    for index1 in range(len(allpoints)):
        #遍历每个marker
        for index2 in range(len(allpoints[index1])):
            #遍历xyz
            for index3 in range(len(allpoints[index1][index2])):
                #遍历xyz内的具体坐标
                allpoints[index1][index2][index3] = optToPixel(allpoints[index1][index2][index3])
    return allpoints

In [13]:
def getScreenSurface(allpoints, indexlist, currentindex):
    #--Step1：求手机平面表达式--
    #取向量31，34计算手机屏幕平面的法向量
    #实时计算平面: 取log(allpoints)中当前数据帧(每行数据)中这几个定位marker数据为该marker的坐标位置
    #indexlist example [fingerIndex,marker1index,marker2index,marker3index,marker4index]=[5, 2, 3, 4, 1]
    marker3 = [allpoints[indexlist[3]-1][0][currentindex], allpoints[indexlist[3]-1][1][currentindex], allpoints[indexlist[3]-1][2][currentindex]]
    marker4 = [allpoints[indexlist[4]-1][0][currentindex], allpoints[indexlist[4]-1][1][currentindex], allpoints[indexlist[4]-1][2][currentindex]]
    marker1 = [allpoints[indexlist[1]-1][0][currentindex], allpoints[indexlist[1]-1][1][currentindex], allpoints[indexlist[1]-1][2][currentindex]]
    
    #求向量31，34的外积
    #向量31=[x1-x3,y1-y3,z1-z3]
    #向量34=[x4-x3,y4-y3,z4-z3]
    vector31 = np.array([marker1[0]-marker3[0],marker1[1]-marker3[1],marker1[2]-marker3[2]])
    vector34 = np.array([marker4[0]-marker3[0],marker4[1]-marker3[1],marker4[2]-marker3[2]])
    
    #手机平面法向量
    vectorn = np.cross(vector31,vector34)
    
    a = vectorn[0]
    b = vectorn[1]
    c = vectorn[2]
    
    #手机平面方程 ax+by+cz+d = 0
    # d = -(ax+by+cz)
    #利用手机平面上一个点（marker1）坐标，计算该平面表达式常量
    d = -(a*marker1[0]+b*marker1[1]+c*marker1[2])
    #返回平面方程的4个参数
    return [[a,b,c,d],[vectorn]]


In [14]:
def getDistance(p, surfaceIndex):
    #--Step2：求点到手机平面的距离，即z1--
    #print(surfaceIndex)
    a,b,c,d = surfaceIndex[0],surfaceIndex[1],surfaceIndex[2],surfaceIndex[3]
    x,y,z = p[0],p[1],p[2]
    dis = abs(a*x+b*y+c*z+d)/math.sqrt(a*a+b*b+c*c)
    return dis
def getZ1(dis,p,vectorn,O):
    xo,yo,zo = O[0],O[1],O[2]
    xp,yp,zp = p[0],p[1],p[2]
    vectorOP = [xp-xo,yp-yo,zp-zo]
    
    n1 = vectorOP[0]*vectorn[0]+vectorOP[1]*vectorn[1]+vectorOP[2]*vectorn[2]
    n2 = math.sqrt(vectorOP[0]*vectorOP[0]+vectorOP[1]*vectorOP[1]+vectorOP[2]*vectorOP[2])
    n3 = math.sqrt(vectorn[0]*vectorn[0]+vectorn[1]*vectorn[1]+vectorn[2]*vectorn[2])
   
    cosOP_n = n1/(n2*n3)
    
    if 0<=cosOP_n<=1:
        z1 = dis
    elif -1<=cosOP_n<0:
        z1 = -dis
    return z1

    

In [15]:
def getProjection(p,vectorn,surfaceIndex):
    #--Step3：求点在手机平面上的投影坐标--
    #点在手机平面上的投影坐标(参数为待计算点P坐标(xp,yp,zp)，手机平面法向量(vx,vy,vz)，手机平面公式参数(abcd))
    #1. 点P与其投影点H形成的向量PH公式为：
    #x = xp + vx*t
    #y = yp + vy*t
    #z = zp + vz*t
    #2. 将以上坐标代入手机平面表达式
    #a*(xp + vx*t)+b*(yp + vy*t)+c*(zp + vz*t)+d = 0
    #求 t并带入向量PH公式，即可得到点H的坐标
    #得 t = -(a*xp+b*yp+c*zp+d)/(a*vx+b*vy+c*vz)
    a,b,c,d = surfaceIndex[0],surfaceIndex[1],surfaceIndex[2],surfaceIndex[3]
    xp,yp,zp = p[0],p[1],p[2]
    vx,vy,vz = vectorn[0],vectorn[1],vectorn[2]
    
    t = -(a*xp+b*yp+c*zp+d)/(a*vx+b*vy+c*vz)
    
    x = xp + vx*t
    y = yp + vy*t
    z = zp + vz*t
    
    return [x,y,z]
    

In [16]:
def getOABC(allpoints,indexlist,currentindex):
    #p1为点P在手机平面上的投影点
    
    #实时计算：取数据log(allpoints)中，当前帧的marker位置
    m3 = [allpoints[indexlist[3]-1][0][currentindex], allpoints[indexlist[3]-1][1][currentindex], allpoints[indexlist[3]-1][2][currentindex]]
    m4 = [allpoints[indexlist[4]-1][0][currentindex], allpoints[indexlist[4]-1][1][currentindex], allpoints[indexlist[4]-1][2][currentindex]]
    m1 = [allpoints[indexlist[1]-1][0][currentindex], allpoints[indexlist[1]-1][1][currentindex], allpoints[indexlist[1]-1][2][currentindex]]
    m2 = [allpoints[indexlist[2]-1][0][currentindex], allpoints[indexlist[2]-1][1][currentindex], allpoints[indexlist[2]-1][2][currentindex]]
    
    #--Step4：求点在手机平面上的投影在手机平面坐标系上的x1,y1--
    #求屏幕左上角点O(xo,yo,zo)，左下角点B(xb,yb,zb)，右上角点A(xa,ya,za)坐标
    #点O和点B在marker1(x1,y1,z1)到marker3(x3,y3,z3)的延长线上
    #1. 求marker1到marker3的直线表达式
    x1,y1,z1 = m1[0],m1[1],m1[2]
    x3,y3,z3 = m3[0],m3[1],m3[2]
    
    #marker1到marker3的直线表达式为
    #x = x3 + (x1-x3)*t
    #y = y3 + (y1-y3)*t
    #z = z3 + (z1-z3)*t
    
    #2. 已知屏幕左上角点O与marker3距离为23mm，带入距离公式
    disOmarker3 = mmTopixel(23)
    to = disOmarker3/math.sqrt((x1-x3)*(x1-x3)+(y1-y3)*(y1-y3)+(z1-z3)*(z1-z3))
    xo = x3 + (x3-x1)*to
    yo = y3 + (y3-y1)*to
    zo = z3 + (z3-z1)*to
    
    #3. 已知屏幕左下角点B与marker3距离为23+113mm，带入距离公式
    disBmarker3 = mmTopixel(136)
    tb = disBmarker3/math.sqrt((x1-x3)*(x1-x3)+(y1-y3)*(y1-y3)+(z1-z3)*(z1-z3))
    xb = x3 + (x3-x1)*tb
    yb = y3 + (y3-y1)*tb
    zb = z3 + (z3-z1)*tb
    
    #4. 求marker2到marker4的直线表达式
    x2,y2,z2 = m2[0],m2[1],m2[2]
    x4,y4,z4 = m4[0],m4[1],m4[2]
    
    #marker1到marker3的直线表达式为
    #x = x4 + (x2-x4)*t
    #y = y4 + (y2-y4)*t
    #z = z4 + (z2-z4)*t
    
    #5. 已知屏幕右上角点A与marker4距离为23mm，带入距离公式
    disAmarker4 = mmTopixel(23)
    ta = disAmarker4/math.sqrt((x2-x4)*(x2-x4)+(y2-y4)*(y2-y4)+(z2-z4)*(z2-z4))
    xa = x4 + (x4-x2)*ta
    ya = y4 + (y4-y2)*ta
    za = z4 + (z4-z2)*ta
    
    #6. 已知屏幕右下角点C与marker4距离为23+113mm，带入距离公式
    disCmarker4 = mmTopixel(136)
    tc = disCmarker4/math.sqrt((x2-x4)*(x2-x4)+(y2-y4)*(y2-y4)+(z2-z4)*(z2-z4))
    xc = x4 + (x4-x2)*tc
    yc = y4 + (y4-y2)*tc
    zc = z4 + (z4-z2)*tc
    
    #返回值为手机屏幕O,A,B,C点的坐标       

    return [[xo,yo,zo],[xa,ya,za],[xb,yb,zb],[xc,yc,zc]]
    
    
def getX1Y1(O,A,B,C,p1):
    
    #求点P在手机平面上投影P1距离手机屏幕上边缘OA之间的距离(y1)和左边缘OB之间的距离(x1)
    #利用海伦公式求三角形OAP1和OBP1的面积S=√[p(p-a)(p-b)(p-c)],其中p=(a+b+c)/2
    #所以某条边x上的高h可以是xh/2=S, 所以h=2S/x,其中x就是高所在边的长度,即x=a或b或c.
    xp1,yp1,zp1 = p1[0],p1[1],p1[2]
    xo,yo,zo = O[0],O[1],O[2]
    xa,ya,za = A[0],A[1],A[2]
    xb,yb,zb = B[0],B[1],B[2]
    xc,yc,zc = C[0],C[1],C[2]
    
    #求OA, AP1, OP1 (三角形OAP1)和OB, BP1, OP1 (三角形OBP1)长度
    OA = math.sqrt((xo-xa)*(xo-xa)+(yo-ya)*(yo-ya)+(zo-za)*(zo-za))
    AP1 = math.sqrt((xp1-xa)*(xp1-xa)+(yp1-ya)*(yp1-ya)+(zp1-za)*(zp1-za))
    OP1 = math.sqrt((xp1-xo)*(xp1-xo)+(yp1-yo)*(yp1-yo)+(zp1-zo)*(zp1-zo))
    OB = math.sqrt((xo-xb)*(xo-xb)+(yo-yb)*(yo-yb)+(zo-zb)*(zo-zb))
    BP1 = math.sqrt((xp1-xb)*(xp1-xb)+(yp1-yb)*(yp1-yb)+(zp1-zb)*(zp1-zb))
    #海伦公式求三角形OAP1和OBP1的面积sOAP1和sOBP1
    pOAP1 = (OA+AP1+OP1)/2
    pOBP1 = (OB+BP1+OP1)/2
    sOAP1 = math.sqrt(pOAP1*int(pOAP1-OA)*int(pOAP1-AP1)*int(pOAP1-OP1))
    sOBP1 = math.sqrt(pOBP1*int(pOBP1-OB)*int(pOBP1-BP1)*int(pOBP1-OP1))
    #求点P1到OA和OB的距离，即三角形OAP1和OBP1分别在边OA和OB上的高hOA和hOB
    hOA = 2*sOAP1/OA
    hOB = 2*sOBP1/OB
    
    #确定手机屏幕坐标[x1,y1]的符号，即判断投影点P1在矩形OABC中:
    #OA以上(y1<0),OA以下(y1>0),OB以左(x1<0),OB以右(x1>0)
    #计算vectorBP1和vectorBC，vectorAP1和vectorAC的夹角cos值
    #cos值在[0,1]范围内则夹角小于90°，对应x1或y1值为正值
    #cos值在[-1,0]范围内则夹角大于90°，对应x1或y1值为负值
    vectorBP1 = [int(xp1-xb),int(yp1-yb),int(zp1-zb)]
    vectorBC = [int(xc-xb),int(yc-yb),int(zc-zb)]
    
    vectorAP1 = [int(xp1-xa),int(yp1-ya),int(zp1-za)]
    vectorAC = [int(xc-xa),int(yc-ya),int(zc-za)]
    
    n1 = vectorBP1[0]*vectorBC[0]+vectorBP1[1]*vectorBC[1]+vectorBP1[2]*vectorBC[2]
    n2 = math.sqrt(vectorBP1[0]*vectorBP1[0]+vectorBP1[1]*vectorBP1[1]+vectorBP1[2]*vectorBP1[2])
    n3 = math.sqrt(vectorBC[0]*vectorBC[0]+vectorBC[1]*vectorBC[1]+vectorBC[2]*vectorBC[2])
    
    m1 = vectorAP1[0]*vectorAC[0]+vectorAP1[1]*vectorAC[1]+vectorAP1[2]*vectorAC[2]
    m2 = math.sqrt(vectorAP1[0]*vectorAP1[0]+vectorAP1[1]*vectorAP1[1]+vectorAP1[2]*vectorAP1[2])
    m3 = math.sqrt(vectorAC[0]*vectorAC[0]+vectorAC[1]*vectorAC[1]+vectorAC[2]*vectorAC[2])
    
    cosBP1_BC = n1/(n2*n3)
    cosAP1_AC = m1/(m2*m3)
    #print(cosBP1_BC)
    #print(cosAP1_AC)
    
    if 0<=cosBP1_BC<=1:
        x1 = hOB
    elif -1<=cosBP1_BC<=0:
        x1 = -hOB

    
    if 0<=cosAP1_AC<=1:
        y1 = hOA
    elif -1<=cosAP1_AC<=0:
        y1 = -hOA
    
    return [x1,y1]
    

In [17]:
def show2D(allpoints):
    import numpy as np
    import matplotlib.pyplot as plt
    fig = plt.figure()
    ax = fig.add_subplot(111)
    #设置标题
    ax.set_title('points mapped on screen')
    #设置X轴标签
    plt.xlabel('X')
    #设置Y轴标签
    plt.ylabel('Y')
    
    #画散点图
    ax.invert_yaxis()
    ax.scatter(allpoints[0][0],allpoints[0][1], c='y')
    ax.scatter(allpoints[1][0],allpoints[1][1], c='r')
    ax.scatter(allpoints[2][0],allpoints[2][1], c='g')
    ax.scatter(allpoints[3][0],allpoints[3][1], c='b')
    ax.scatter(allpoints[4][0],allpoints[4][1], c='c')
       
    #显示所画的图
    plt.show()

In [18]:
def getCalibrationPos(fingerp1,buttonthreshold):
    #返回当前手指坐标所在按钮的编号
    x = fingerp1[0]
    y = fingerp1[1]
    #z = fingerp1[2]
    button1 = [100,700]
    button2 = [1340,700]
    button3 = [100,1500]
    button4 = [1340,1500]
    buttonsize = 200
    #因为optiTrack的marker粘在指甲上，与实际点击按钮的手指位置有偏差
    #因此设定buttonthreshold，只要finger在button之外buttonthreshold距离范围内，就将其判定为在点击按钮
    if abs(x-button1[0])<100+buttonthreshold and abs(y-button1[1])<100+buttonthreshold:
        return [1,button1]
    elif abs(x-button2[0])<100+buttonthreshold and abs(y-button2[1])<100+buttonthreshold:
        return [2,button2]
    elif abs(x-button3[0])<100+buttonthreshold and abs(y-button3[1])<100+buttonthreshold:
        return [3,button3]
    elif abs(x-button4[0])<100+buttonthreshold and abs(y-button4[1])<100+buttonthreshold:
        return [4,button4]
    else:
        return [0,0]
        

In [19]:
def getCalibrationLog(allpointsmartphone, optitracklog, indexlist, hand, buttonthreshold,thresholdz):
    # 找到点击按钮的时刻，记录用于时间校正的数据
    pointinglog = []
    #点击校正按钮的时刻
    time = []
    #点击的按钮编号
    button = []
    #记录点击动作时手指的坐标位置
    optifingerposx = []
    optifingerposy = []
    optifingerposz = []
    
    if hand==1:
        fingerpoint = allpointsmartphone[indexlist[0]-1]
    elif hand==2:
        finger1point = allpointsmartphone[indexlist[0]-1]
        finger2point = allpointsmartphone[indexlist[5]-1]
        
    #按下按钮行为阈值，在此阈值以内的手指高度最低点即为点击时刻手指的位置
    #在每帧计算平面方程的条件下，需要设定thresholdz = 80
    flagpressing = 0
    flag1pressing = 0
    flag2pressing = 0
    tempz = 10000
    temp1z = 10000
    temp2z = 10000
    if hand==1:
        for index in range(len(fingerpoint[0])):

            if flagpressing==1:
                #在点击按钮的状态下（flagpressing=1）保留高度z最小的手指坐标点信息和在整体数据中的index
                #因为z值为负，因此判断高度小于之前记录的数据时用>判断

                if -fingerpoint[2][index]<tempz:
                    print(-fingerpoint[2][index])
                    tempx = fingerpoint[0][index]
                    tempy = fingerpoint[1][index]
                    tempz = -fingerpoint[2][index]
                    tempindex = index

            if -fingerpoint[2][index]<thresholdz and flagpressing==0:
                #检查手指坐标数据中z分量在每行中的值
                #如果距离手机屏幕小于thresholdz，且flagpressing==0则识别为开始按下按钮flagpressing = 1
                print('-----------press-------------')
                flagpressing = 1

            if -fingerpoint[2][index]>thresholdz and flagpressing==1:
                print('-----------release-------------')

                #将按下按钮并抬起手指的瞬时时刻记录为时间轴校正的时刻
                flagpressing = 0

                #设置在按钮及按钮之外20像素范围内为识别按钮点击行为的范围，返回按钮index (1,2,3,4)
                #返回值为0时，即未点击任何按钮
                #用点击按钮过程中的最低点(tempx,tempy,tempz)计算当前点击的按钮
                buttonInfo = getCalibrationPos([tempx,tempy,tempz],buttonthreshold)

                if buttonInfo[0]!=0 and -10<tempz<thresholdz:
                    if button==[1, 2, 3, 4]:
                        print('-------------------finish recognition of calibration button press------------------')
                        #如果已经识别到了4个矫正按钮的点击，则停止计算
                        break
                    #如果成功识别点击的按钮，则记录该时刻tempindex的信息
                    time.append(int(float(optitracklog[tempindex][1])*1000))
                    button.append(buttonInfo[0])
                    optifingerposx.append(fingerpoint[0][tempindex])
                    optifingerposy.append(fingerpoint[1][tempindex])
                    optifingerposz.append(-fingerpoint[2][tempindex])
                    print('Press time: '+str(optitracklog[tempindex][1]))
                    print('Recorded distance: '+str(-fingerpoint[2][tempindex]))
                    print('Button number: '+str(button))
                    print('Finger pos from optitrack: '+str([tempx,tempy,tempz]))

                #抬起按钮并记录完所有数据之后，重置手指点击的最低坐标
                tempz = 10000
    elif hand==2:
        #遍历 hand1
        for index in range(len(finger1point[0])):
            if flag1pressing==1:
                #在点击按钮的状态下（flag1pressing=1）保留高度z最小的手指坐标点信息和在整体数据中的index
                #因为z值为负，因此判断高度小于之前记录的数据时用>判断

                if -finger1point[2][index]<temp1z:
                    print(-finger1point[2][index])
                    temp1x = finger1point[0][index]
                    temp1y = finger1point[1][index]
                    temp1z = -finger1point[2][index]
                    temp1index = index

            if -finger1point[2][index]<thresholdz and flag1pressing==0:
                #检查手指坐标数据中z分量在每行中的值
                #如果距离手机屏幕小于thresholdz，且flagpressing==0则识别为开始按下按钮flagpressing = 1
                print('----press 1----')
                flag1pressing = 1

            if -finger1point[2][index]>thresholdz and flag1pressing==1:
                print('----release 1----')

                #将按下按钮并抬起手指的瞬时时刻记录为时间轴校正的时刻
                flag1pressing = 0

                #设置在按钮及按钮之外20像素范围内为识别按钮点击行为的范围，返回按钮index (1,2,3,4)
                #返回值为0时，即未点击任何按钮
                #用点击按钮过程中的最低点(temp1x,temp1y,temp1z)计算当前点击的按钮
                buttonInfo = getCalibrationPos([temp1x,temp1y,temp1z],buttonthreshold)

                if buttonInfo[0]!=0 and -10<temp1z<thresholdz:
                    if button==[1,2, 3, 4]:
                        print('-------------------finish recognition of calibration button press------------------')
                        #如果已经识别到了4个矫正按钮的点击，则停止计算
                        break
                    #如果成功识别点击的按钮，则记录该时刻tempindex的信息
                    time.append(int(float(optitracklog[temp1index][1])*1000))
                    button.append(buttonInfo[0])
                    optifingerposx.append(finger1point[0][temp1index])
                    optifingerposy.append(finger1point[1][temp1index])
                    optifingerposz.append(-finger1point[2][temp1index])
                    print('Press time: '+str(optitracklog[temp1index][1]))
                    print('Recorded distance: '+str(-finger1point[2][temp1index]))
                    print('Button number: '+str(button))
                    print('Finger pos from optitrack: '+str([temp1x,temp1y,temp1z]))

                #抬起按钮并记录完所有数据之后，重置手指点击的最低坐标
                temp1z = 10000
            #遍历 hand2
            if flag2pressing==1:
                #在点击按钮的状态下（flag2pressing=1）保留高度z最小的手指坐标点信息和在整体数据中的index
                #因为z值为负，因此判断高度小于之前记录的数据时用>判断

                if -finger2point[2][index]<temp2z:
                    print('                              '+str(-finger2point[2][index]))
                    temp2x = finger2point[0][index]
                    temp2y = finger2point[1][index]
                    temp2z = -finger2point[2][index]
                    temp2index = index

            if -finger2point[2][index]<thresholdz and flag2pressing==0:
                #检查手指坐标数据中z分量在每行中的值
                #如果距离手机屏幕小于thresholdz，且flagpressing==0则识别为开始按下按钮flagpressing = 1
                print('                              ----press 2----')
                flag2pressing = 1

            if -finger2point[2][index]>thresholdz and flag2pressing==1:
                print('                              ----release 2----')

                #将按下按钮并抬起手指的瞬时时刻记录为时间轴校正的时刻
                flag2pressing = 0

                #设置在按钮及按钮之外20像素范围内为识别按钮点击行为的范围，返回按钮index (1,2,3,4)
                #返回值为0时，即未点击任何按钮
                #用点击按钮过程中的最低点(temp2x,temp2y,temp2z)计算当前点击的按钮
                buttonInfo = getCalibrationPos([temp2x,temp2y,temp2z],buttonthreshold)

                if buttonInfo[0]!=0 and -10<temp2z<thresholdz:
                    if button==[1,2, 3, 4]:
                        print('-------------------finish recognition of calibration button press------------------')
                        #如果已经识别到了4个矫正按钮的点击，则停止计算
                        break
                    #如果成功识别点击的按钮，则记录该时刻tempindex的信息
                    time.append(int(float(optitracklog[temp2index][1])*1000))
                    button.append(buttonInfo[0])
                    optifingerposx.append(finger2point[0][temp2index])
                    optifingerposy.append(finger2point[1][temp2index])
                    optifingerposz.append(-finger2point[2][temp2index])
                    print('                              Press time: '+str(optitracklog[temp2index][1]))
                    print('                              Recorded distance: '+str(-finger2point[2][temp2index]))
                    print('                              Button number: '+str(button))
                    print('                              Finger pos from optitrack: '+str([temp2x,temp2y,temp2z]))
                #抬起按钮并记录完所有数据之后，重置手指点击的最低坐标
                temp2z = 10000

    pointinglog.append(time)
    pointinglog.append(button)
    pointinglog.append(optifingerposx)
    pointinglog.append(optifingerposy)
    pointinglog.append(optifingerposz)

    return pointinglog
        

In [20]:
def writeCalibrationOptiTrack(pointinglog):
    import pandas as pd

    dataframe = pd.DataFrame({'time':pointinglog[0], 'button':pointinglog[1], 'x':pointinglog[2] , 'y': pointinglog[3], 'z': pointinglog[4]})
    cols=['time', 'button', 'x', 'y', 'z'] #按列名顺序输出
    dataframe = dataframe.ix[:,cols]
    filename = directory+str(currentid)+'_'+str(currentblock)+'_calibrationOptiTrack.csv'
    dataframe.to_csv(filename,index=False,sep=',')

In [21]:
def getStartEndsmartphonesystime():
    keyevent = readKeyevent()
    #找到点击每个句子第一个和最后一个字母的时刻
    #返回数组[['sentencestartsystime', 'sentenceendsystime', 'sentence_n'],[...]]
    trialstartend = []
    startend = []
    for i in range(len(keyevent)):
        if i==0:
            startend.append(keyevent[i][0])
            currentsentence = int(keyevent[i][3])
        elif i==len(keyevent)-1:
            startend.append(keyevent[i][0])
            startend.append(keyevent[i][3])
            trialstartend.append(startend)
        else:
            if int(keyevent[i][3])!=currentsentence or int(keyevent[i][4])==0:
                currentsentence = int(keyevent[i][3])
                startend.append(keyevent[i-1][0])
                startend.append(keyevent[i-1][3])
                trialstartend.append(startend)
                startend = []
                startend.append(keyevent[i][0])
    return trialstartend

In [22]:
def rejectOutliers(data):
    u = np.mean(data)
    s = np.std(data)
    filtered = [e for e in data if (u - 2 * s < e < u + 2 * s)]
    if len(data)!=len(filtered):
        print('outlier rejected:')
        print(data)
        print(filtered)
    return filtered

In [23]:
def optiTimetoSmartphone(pointinglog,smartphonelog,optitime,trialstartend,enterlog):
    
    #将optiTrack的时间转换成以smartphone为基准的时间
    #手机和optiTrack检测到的按钮点击事件列表
    smartphonebuttonpress = []
    optibuttonpress = []
    #手机和optiTrack检测到的按钮点击时间
    smartphonetimelist = []
    optitimelist = []
    #手机和optiTrack检测到的同一按钮点击时间差
    timeoffsetlist = []
    for line in smartphonelog:
        smartphonebuttonpress.append(int(line[3]))
        smartphonetimelist.append(int(line[0]))
    optibuttonpress = pointinglog[1]
    optitimelist = pointinglog[0]
    if smartphonebuttonpress==optibuttonpress:
        for i in range(len(smartphonebuttonpress)):
            timeoffsetlist.append(float(optitimelist[i])-float(smartphonetimelist[i]))

    #取所有点击事件的时间差平均值进行时间轴对准
    smatphonesystime = optitime - np.mean(rejectOutliers(timeoffsetlist))
    
    #将smartphone系统时间换算成trialtime
    if smatphonesystime<=int(enterlog[0][0]):
        temptrialtime = smatphonesystime - int(trialstartend[0][0])
        sentence_n = int(trialstartend[0][2])
    elif smatphonesystime>=int(enterlog[len(enterlog)-1][0]):
        temptrialtime = smatphonesystime - int(trialstartend[len(trialstartend)-1][0])
        sentence_n = int(trialstartend[len(trialstartend)-1][2])
    else:
        for index in range(len(enterlog)):
            if index==0:
                continue
            elif int(enterlog[index-1][0])<=smatphonesystime<=int(enterlog[index][0]):
                temptrialtime = smatphonesystime-int(trialstartend[index][0])
                sentence_n = int(trialstartend[index][2])
    return [int(temptrialtime),sentence_n]

In [24]:
#101_2
#trialstartend = [['1535714864138', '1535714871155', '08'], ['1535714878004', '1535714893602', '33'], ['1535714900706', '1535714906473', '05'], ['1535714914089', '1535714921926', '02'], ['1535714928610', '1535714941692', '47'], ['1535714948978', '1535714958649', '61'], ['1535714966365','1535714970403','68',],['1535714994226','1535715003392', '49'], ['1535715009424', '1535715015536', '72'], ['1535715022039', '1535715032097', '11'], ['1535715098111', '1535715105299', '70'], ['1535715112382', '1535715119671', '52'], ['1535715126402', '1535715134032', '09'], ['1535715141240', '1535715149594', '18'], ['1535715157060', '1535715171504', '51'], ['1535715178439', '1535715186572', '60'], ['1535715193183', '1535715199058', '41'], ['1535715206024', '1535715213778', '62'], ['1535715220697', '1535715229669', '44'], ['1535715235810', '1535715242050', '66']]

In [25]:
def getFingerCSV(optitracklog,pointinglog,smartphonelog,allpointsmartphon,indexlist,uid,block):
    #optitime,id,block,sentence_n,trialtime,hand,x1,y1,z1,x2,y2,z2
    optitimelist = []
    uidlist = []
    blocklist = []
    sentencelist = []
    trialtimelist = []
    handlist = []
    x1list = []
    y1list = []
    z1list = []
    x2list = []
    y2list = []
    z2list = []
    
    trialstartend = getStartEndsmartphonesystime()
    enterlog = readEnterlog()
    
    if len(optitracklog[0])==17:
        #通过marker数计算实验使用的手数量
        hand = 1
    elif len(optitracklog[0])==20:
        hand = 2
    else:
        print('Strange marker number')
    for i in range(len(optitracklog)):
        uidlist.append(uid)
        blocklist.append(block)
        
        #OptiTrack时间轴
        opttime = int(float(optitracklog[i][1])*1000)
        optitimelist.append(opttime)
        #转换成smartphone时间轴
        timeandsentence = optiTimetoSmartphone(pointinglog,smartphonelog,opttime,trialstartend,enterlog)
        trialtimelist.append(timeandsentence[0])
        sentencelist.append(timeandsentence[1])
        handlist.append(hand)
        #visualization的背景图片左上角到手机屏幕左上角之间在x和y轴方向上的偏差为500，200
        x1list.append(allpointsmartphone[indexlist[0]-1][0][i]+500)
        y1list.append(allpointsmartphone[indexlist[0]-1][1][i]+200)
        z1list.append(-allpointsmartphone[indexlist[0]-1][2][i])
        if hand==1:
            #单手情况hand2的x,y,z置0
            x2list.append(0)
            y2list.append(0)
            z2list.append(0)
        elif hand==2:
            x2list.append(allpointsmartphone[indexlist[5]-1][0][i]+500)
            y2list.append(allpointsmartphone[indexlist[5]-1][1][i]+200)
            z2list.append(-allpointsmartphone[indexlist[5]-1][2][i])
            
    dataframe = pd.DataFrame({'optitime':optitimelist,'id':uidlist,'block':blocklist
                              ,'sentence_n':sentencelist,'trialtime':trialtimelist,'hand':handlist
                              ,'x1':x1list,'y1':y1list,'z1':z1list,'x2':x2list,'y2':y2list,'z2':z2list})
    
    cols=['optitime','id','block','sentence_n','trialtime','hand','x1','y1','z1','x2','y2','z2'] #按列名顺序输出
    dataframe = dataframe.ix[:,cols]
    filename = directory+'finger_'+str(uid)+'_'+str(block)+'.csv'
    dataframe.to_csv(filename,index=False,sep=',')

In [49]:

currentid = 118
currentblock = 1
buttonthreshold = 200
thresholdz = 150

directory = 'F:/Eye-Hand Coordination-JXH/4 Experiment data/2 Edited data/'+str(currentid)+'/'

print(directory)

F:/Eye-Hand Coordination-JXH/4 Experiment data/2 Edited data/118/


In [50]:
#main
#读手机上的log信息
smartphonelog = addtrialtime(readSmartphone())
#读OptiTrack的数据信息
optitracklogInfo = readOptiTrack()
optitracklog = optitracklogInfo[1]
hand = optitracklogInfo[0]
#将OptiTrack数据信息转换成像素为单位的坐标，放入list中
allpoints = transferOptiToPixel(generateAllPoints(optitracklog,hand))
#从OptiTrack数据中识别4个定位marker和手指marker在list中的索引位置
indexlist = getIndex(allpoints,hand)


Finger1Index = 5
Finger2Index = 6
每OptiTrack单位对应mm数为0.04332967614360138
每OptiTrack单位对应手机pixel数为0.9903925975680314


In [51]:
#计算所有坐标在手机坐标系下的坐标值并放入allpointsmartphone
allpointsmartphone = []
for point in allpoints:
    #判断当前的点是否是fingermarker，是则正常计算，否则将y值取负
    onepoint = []
    pointx1 = []
    pointy1 = []
    pointz1 = []
    for i in range(len(point[0])):
        #实时计算：4个定位marker形成的平面（手机平面）表达式中的ax+by+cz+d=0中的abcd和平面的法向量
        screenSurfaceInfo = getScreenSurface(allpoints, indexlist,i)
        surfaceIndex = screenSurfaceInfo[0]
        vectorn = screenSurfaceInfo[1][0]
        
        p = [point[0][i],point[1][i],point[2][i]]
        #得到空间中任意一个点p在手机平面上的投影坐标p1
        p1 = getProjection(p,vectorn,surfaceIndex)
        #求屏幕4个角落点O，A，B，C的坐标
        OABC = getOABC(allpoints,indexlist,i)
        #求点在手机坐标系下的x1y1
        x1y1 = getX1Y1(OABC[0],OABC[1],OABC[2],OABC[3],p1)
        pointx1.append(x1y1[0])
        pointy1.append(x1y1[1])
        #计算空间中任意一个点p到手机平面之间的距离（即转换成手机坐标系之后的z1）
        dis = getDistance(p, surfaceIndex)
        z1 = getZ1(dis,p,vectorn,OABC[0])
        pointz1.append(z1)
    #print(pointz1)
    onepoint.append(pointx1)
    onepoint.append(pointy1)
    onepoint.append(pointz1)
    allpointsmartphone.append(onepoint)

print('Finished...')
    
#showAllpoints(allpoints,hand)
#showAllpoints(allpointsmartphone,hand)
#因为把右手定则坐标系转化成了左手定则坐标系，所以z1坐标变成了负值
#这个变化不影响计算，实际判断z1时加个-号就可以正常使用

Finished...


In [52]:
#生成OptiTrack方面的时间同步文件，并写入calibrationOptiTrack.csv
pointinglog = getCalibrationLog(allpointsmartphone, optitracklog, indexlist,hand,buttonthreshold,thresholdz)
#(only for 127-1)pointinglog = [[7833, 8766, 10016, 11066], [1, 2, 3, 4], [192.4722034106361, 1513.9613146917502, 213.91404277110522, 1540.8377443798759], [634.8893741745496, 628.8099044726333, 1484.5898100219495, 1440.2885809744507], [80.154497997938549, 85.028228292947219, 58.327931471340484, 63.726738779523643]]
writeCalibrationOptiTrack(pointinglog)

                              ----press 2----
                              132.148656111
                              116.350276613
                              98.1768491063
                              85.7925634353
                              77.1626317971
                              66.9341281705
                              50.764234232
                              43.2095670843
                              32.6146285969
                              30.9507145615
                              30.4043346351
                              24.7384151658
                              19.0037637917
                              15.689456135
                              8.26120055171
                              7.18873159397
                              5.23821442739
                              2.14630357664
                              -11.4642248134
                              -19.4999698219
                              -25.753077513
                              

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


In [53]:

#(only for 133-2)smartphonelog = [['1536819970287', '1322.5781', '701.875', '2', 568], ['1536819971030', '133.94531', '1521.875', '3', 1311], ['1536819971550', '1310.2734', '1580.0', '4', 1831]]
getFingerCSV(optitracklog,pointinglog,smartphonelog,allpointsmartphone,indexlist,currentid,currentblock)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:57: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [32]:
getStartEndsmartphonesystime()

[['1536306462976', '1536306472071', '47'],
 ['1536306478666', '1536306489053', '18'],
 ['1536306495083', '1536306508900', '44'],
 ['1536306514920', '1536306523482', '62'],
 ['1536306529353', '1536306532107', '37'],
 ['1536306538199', '1536306543289', '21'],
 ['1536306548731', '1536306551773', '01'],
 ['1536306557970', '1536306562103', '02'],
 ['1536306568879', '1536306572083', '53'],
 ['1536306579583', '1536306583108', '38'],
 ['1536306614472', '1536306636570', '999'],
 ['1536306642389', '1536306647997', '20'],
 ['1536306654781', '1536306661144', '23'],
 ['1536306667503', '1536306676413', '51'],
 ['1536306682760', '1536306691006', '33'],
 ['1536306696621', '1536306703052', '15'],
 ['1536306708839', '1536306712339', '36'],
 ['1536306719511', '1536306725160', '06'],
 ['1536306731545', '1536306734945', '61'],
 ['1536306746933', '1536306755334', '75'],
 ['1536306761337', '1536306773200', '999']]

In [99]:
#test-生成CSV文件到R中看3D效果图

import pandas as pd

col_x = []
col_y = []
col_z = []
            
for point in allpointsmartphone:
    for i in range(int(len(point[0])/5)):
        col_x.append(point[0][i])
        col_y.append(point[1][i])
        col_z.append(point[2][i])

dataframe = pd.DataFrame({'x':col_x, 'y':col_y, 'z':col_z})
cols=['x', 'y', 'z'] #按列名顺序输出
dataframe = dataframe.ix[:,cols]
filename = 'allpointsmartphone.csv'
dataframe.to_csv(filename,index=False,sep=',')

C:\Users\jiangx3\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:17: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [101]:
#test-生成CSV文件到R中看3D效果图

import pandas as pd

col_x = []
col_y = []
col_z = []

point = allpointsmartphone[indexlist[0]-1]
for i in range(int(len(point[0])/5)):
    col_x.append(point[0][i])
    col_y.append(point[1][i])
    col_z.append(point[2][i])

dataframe = pd.DataFrame({'x':col_x, 'y':col_y, 'z':col_z})
cols=['x', 'y', 'z'] #按列名顺序输出
dataframe = dataframe.ix[:,cols]
filename = 'allpointsmartphone.csv'
dataframe.to_csv(filename,index=False,sep=',')

C:\Users\jiangx3\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:17: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
